In [1]:
import scipy.io as io
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Flatten, MaxPooling2D , Dropout
from keras.initializers import RandomUniform
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.utils import to_categorical
from keras.initializers import RandomUniform

Using TensorFlow backend.


<h1> Model, Data Preprocessing and other functions

In [2]:
def get_model(inputshape,kernel_size1,kernel_size2,activationfunc,kernelinitializer,no_of_filters,pool_size,dense_layer_nodes1,num_classes):
    model = Sequential()
    model.add(Conv2D(no_of_filters, kernel_size1, activation=activationfunc, input_shape=inputshape,kernel_initializer=kernelinitializer,bias_initializer='zeros'))
    model.add(MaxPooling2D(pool_size = pool_size))
    model.add(Flatten())
    model.add(Dense(dense_layer_nodes1,activation=activationfunc,kernel_initializer=kernelinitializer))
    model.add(Dense(num_classes,activation = 'softmax'))
    return model

In [3]:
def data_preprocess(data,labels):
    num_classes = np.max(labels)
    data_in_classes = {x:[] for x in range(1,num_classes+1)}
    (imgdm1,imgdm2,bands) = data.shape
    for i in range(imgdm1):
        for j in range(imgdm2):
            l = labels[i][j]
            if l!=0:
                data_in_classes[l].append(np.array(data[i][j]))
    #printfunc(data_in_classes)
    y = []
    X = []
    for i in range(1,num_classes+1):
        for j in range(len(data_in_classes[i])):
            X.append(data_in_classes[i][j])
            y.append(i)
    X = np.array(X)
    y = np.array(y)
    X = normalize(X,axis=1,return_norm=False)
    return X,y

<h1> A separate Preprocessing Method for Indian Pines is required as we eliminate some classes which have very little sample

In [4]:
def data_preprocess_IndiaPines(data,labels):
    num_classes = np.max(labels)
    data_in_classes = {x:[] for x in range(1,num_classes+1)}
    (imgdm1,imgdm2,bands) = data.shape
    for i in range(imgdm1):
        for j in range(imgdm2):
            l = labels[i][j]
            if l!=0:
                data_in_classes[l].append(np.array(data[i][j]))
    y = []
    X = []
    for i in range(1,17):
        if(len(data_in_classes[i])>480):
            for j in range(len(data_in_classes[i])):
                X.append(data_in_classes[i][j])
                y.append(i)
    X = np.array(X)
    y = np.array(y)
    X = normalize(X,axis=1,return_norm=False)
    return X,y

In [5]:
def load_data_labels(path,data_file_name):
    return io.loadmat(path)[data_file_name] 

In [6]:
def split_data(X,y,test_size,random_state,no_bands):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_state)
    X_test = X_test.reshape(len(X_test),no_bands,1,1)
    X_train = X_train.reshape(len(X_train),no_bands,1,1)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    y_train = y_train[:,1:]
    y_test = y_test[:,1:]
    return X_test,X_train,y_train,y_test

In [7]:
def train(model,optimizer,epoch,batchsize,lossused,X_test,X_train,y_test,y_train):
    model.compile(optimizer=optimizer,loss=lossused,metrics = ['accuracy'])
    filepath="weights.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = epoch,verbose=1,batch_size=batchsize,callbacks=callbacks_list)

<h1> Simulations on the Datasets of the functions

In [8]:
dataset = load_data_labels('PaviaU.mat','paviaU')
labels = load_data_labels('PaviaU_gt.mat','paviaU_gt')
X,y = data_preprocess(dataset,labels)
X_test,X_train,y_train,y_test = split_data(X,y,0.25,42,dataset.shape[2])
inputshape = dataset[0][0].reshape(dataset[0][0].shape[0],1,1).shape
kernelsize1 = np.array([11,1])
kernelsize2 = np.array([3,1])
initializer = RandomUniform(minval=-0.05, maxval=0.05)
filters = 20
pool_size = np.array([3,1])
nodes = 100
classes = 9
model = get_model(inputshape,kernelsize1,kernelsize2,'tanh',initializer,filters,pool_size,nodes,classes)

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 93, 1, 20)         240       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 1, 20)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 620)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               62100     
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 909       
Total params: 63,249
Trainable params: 63,249
Non-trainable params: 0
_________________________________________________________________


In [10]:
train(model,'adam',20,10,'categorical_crossentropy',X_test,X_train,y_test,y_train)

Train on 28659 samples, validate on 14117 samples
Epoch 1/20
28659/28659 [==============================] - 8s 275us/step - loss: 0.7410 - acc: 0.7044 - val_loss: 0.6339 - val_acc: 0.7497
Epoch 2/20
28659/28659 [==============================] - 8s 269us/step - loss: 0.5563 - acc: 0.7862 - val_loss: 0.5075 - val_acc: 0.8134
Epoch 3/20
28659/28659 [==============================] - 8s 272us/step - loss: 0.4706 - acc: 0.8244 - val_loss: 0.4472 - val_acc: 0.8379
Epoch 4/20
28659/28659 [==============================] - 8s 272us/step - loss: 0.4276 - acc: 0.8384 - val_loss: 0.4756 - val_acc: 0.8240
Epoch 5/20
28659/28659 [==============================] - 8s 284us/step - loss: 0.4055 - acc: 0.8474 - val_loss: 0.3752 - val_acc: 0.8614
Epoch 6/20
28659/28659 [==============================] - 8s 273us/step - loss: 0.3918 - acc: 0.8513 - val_loss: 0.4122 - val_acc: 0.8464
Epoch 7/20
28659/28659 [==============================] - 8s 275us/step - loss: 0.3827 - acc: 0.8547 - val_loss: 0.3697 - 

In [11]:
dataset1 = load_data_labels('Salinas_corrected.mat','salinas_corrected')
labels1 = load_data_labels('Salinas_gt.mat','salinas_gt')
X1,y1 = data_preprocess(dataset1,labels1)
X_test1,X_train1,y_train1,y_test1 = split_data(X1,y1,0.25,42,dataset1.shape[2])
inputshape1 = dataset1[0][0].reshape(dataset1[0][0].shape[0],1,1).shape
kernelsize11 = np.array([11,1])
kernelsize22 = np.array([3,1])
initializer1 = RandomUniform(minval=-0.05, maxval=0.05)
filters = 20
pool_size = np.array([3,1])
nodes = 100
classes1 = 16
model1 = get_model(inputshape1,kernelsize11,kernelsize22,'tanh',initializer,filters,pool_size,nodes,classes1)

In [12]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 194, 1, 20)        240       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 1, 20)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               128100    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                1616      
Total params: 129,956
Trainable params: 129,956
Non-trainable params: 0
_________________________________________________________________


In [13]:
train(model1,'adam',20,10,'categorical_crossentropy',X_test1,X_train1,y_test1,y_train1)

Train on 36266 samples, validate on 17863 samples
Epoch 1/20
36266/36266 [==============================] - 16s 453us/step - loss: 0.7845 - acc: 0.7067 - val_loss: 0.4445 - val_acc: 0.8332
Epoch 2/20
36266/36266 [==============================] - 15s 425us/step - loss: 0.4259 - acc: 0.8387 - val_loss: 0.4721 - val_acc: 0.8050
Epoch 3/20
36266/36266 [==============================] - 16s 441us/step - loss: 0.3875 - acc: 0.8500 - val_loss: 0.3880 - val_acc: 0.8434
Epoch 4/20
36266/36266 [==============================] - 16s 434us/step - loss: 0.3619 - acc: 0.8601 - val_loss: 0.3364 - val_acc: 0.8778
Epoch 5/20
36266/36266 [==============================] - 16s 440us/step - loss: 0.3444 - acc: 0.8648 - val_loss: 0.3256 - val_acc: 0.8680
Epoch 6/20
36266/36266 [==============================] - 16s 436us/step - loss: 0.3308 - acc: 0.8675 - val_loss: 0.3227 - val_acc: 0.8801
Epoch 7/20
36266/36266 [==============================] - 17s 460us/step - loss: 0.3218 - acc: 0.8709 - val_loss: 0.

In [14]:
dataset2 = load_data_labels('Indian_pines.mat','indian_pines')
labels2 = load_data_labels('Indian_pines_gt.mat','indian_pines_gt')
X2,y2 = data_preprocess_IndiaPines(dataset2,labels2)
X_test2,X_train2,y_train2,y_test2 = split_data(X2,y2,0.25,42,dataset2.shape[2])
inputshape2 = dataset2[0][0].reshape(dataset2[0][0].shape[0],1,1).shape
kernelsize3 = np.array([11,1])
kernelsize4 = np.array([3,1])
initializer11 = RandomUniform(minval=-0.05, maxval=0.05)
filters = 20
pool_size = np.array([3,1])
nodes = 100
classes2 = np.max(y2)
model2 = get_model(inputshape2,kernelsize3,kernelsize4,'tanh',initializer,filters,pool_size,nodes,classes2)

In [17]:
train(model2,'adam',20,10,'categorical_crossentropy',X_test2,X_train2,y_test2,y_train2)

Train on 5866 samples, validate on 2890 samples
Epoch 1/20
5866/5866 [==============================] - 3s 492us/step - loss: 0.6507 - acc: 0.7532 - val_loss: 0.5555 - val_acc: 0.7969
Epoch 2/20
5866/5866 [==============================] - 3s 455us/step - loss: 0.6357 - acc: 0.7559 - val_loss: 0.6418 - val_acc: 0.7516
Epoch 3/20
5866/5866 [==============================] - 3s 462us/step - loss: 0.6301 - acc: 0.7545 - val_loss: 0.5861 - val_acc: 0.7754
Epoch 4/20
5866/5866 [==============================] - 3s 459us/step - loss: 0.6297 - acc: 0.7601 - val_loss: 0.5646 - val_acc: 0.7862
Epoch 5/20
5866/5866 [==============================] - 3s 466us/step - loss: 0.6215 - acc: 0.7615 - val_loss: 0.6910 - val_acc: 0.7270
Epoch 6/20
5866/5866 [==============================] - 3s 473us/step - loss: 0.6178 - acc: 0.7639 - val_loss: 0.5776 - val_acc: 0.7827
Epoch 7/20
5866/5866 [==============================] - 3s 480us/step - loss: 0.6155 - acc: 0.7629 - val_loss: 0.7736 - val_acc: 0.7000
